In [1]:
from typing import Tuple
from coherence.custom_types import Batch

from tqdm.notebook import tqdm

import jax
import jax.numpy as jnp

import haiku as hk
import optax

from coherence.data import get_data, decimate, normalize, sanitize, get_data_by_class
from coherence.train_with_state import network_and_loss, do_training, update_params, net_accuracy
from coherence.models.cnn import cifar_vgg_11_fn

import coherence.pruning as pr
import coherence.coherence as cr

from coherence.utils import ravel_pytree

from functools import partial


2022-08-02 16:28:10.745338: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
/home/robert/coherence/.env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# load mnist data
train, train_eval, test_eval = get_data("cifar10",batch_size=100,format_fun=sanitize)

dsets_by_class = get_data_by_class("cifar10",batch_size=100,format_fun=sanitize)

# cnn, loss, params
net, xent_loss = network_and_loss(cifar_vgg_11_fn)

key = jax.random.PRNGKey(42)
params, state = net.init(key, next(train)["image"])

2022-08-02 16:30:13.325087: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-02 16:30:13.325152: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


In [11]:
rules = [pr.Rule('batch_norm',1)]
plan = pr.create_plan(params,rules,default_value=None)
default_mask = pr.init_mask(params)
print(plan)

b = pr.apply_where(plan,lambda _:1, params)

{'mlp/~/conv_0/~/batch_norm': {'offset': 1, 'scale': 1}, 'mlp/~/conv_0/~/conv2_d': {'b': None, 'w': None}, 'mlp/~/conv_10/~/batch_norm': {'offset': 1, 'scale': 1}, 'mlp/~/conv_10/~/conv2_d': {'b': None, 'w': None}, 'mlp/~/conv_11/~/batch_norm': {'offset': 1, 'scale': 1}, 'mlp/~/conv_11/~/conv2_d': {'b': None, 'w': None}, 'mlp/~/conv_2/~/batch_norm': {'offset': 1, 'scale': 1}, 'mlp/~/conv_2/~/conv2_d': {'b': None, 'w': None}, 'mlp/~/conv_4/~/batch_norm': {'offset': 1, 'scale': 1}, 'mlp/~/conv_4/~/conv2_d': {'b': None, 'w': None}, 'mlp/~/conv_5/~/batch_norm': {'offset': 1, 'scale': 1}, 'mlp/~/conv_5/~/conv2_d': {'b': None, 'w': None}, 'mlp/~/conv_7/~/batch_norm': {'offset': 1, 'scale': 1}, 'mlp/~/conv_7/~/conv2_d': {'b': None, 'w': None}, 'mlp/~/conv_8/~/batch_norm': {'offset': 1, 'scale': 1}, 'mlp/~/conv_8/~/conv2_d': {'b': None, 'w': None}, 'mlp/~/linear': {'b': None, 'w': None}}
